In [2]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils import data

import numpy as np

import os
import sys

import matplotlib.pyplot as plt

sys.path.append(os.path.abspath('..'))
from LoadRealRunningJumping import *
from LoadSyntheticRunningJumping import *

from GANModels import *

from dataLoader import *

import torch.fft as fft

from EvaluationFunctions import DagharUniclassEvaluation

from sklearn.manifold import TSNE

from sklearn.model_selection import train_test_split

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:55: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


In [3]:
data_path = '../DAGHAR_split_25_10/train/data/'
dataset = daghar_load_dataset(class_name='KuHar_DAGHAR_Multiclass', path = data_path, seq_len=30)

return single class data and labels, class is KuHar_DAGHAR_Multiclass
data shape is (2784, 6, 1, 30)
label shape is (2784,)


In [18]:
a = np.array(dataset[:][0])
b = np.array(dataset[:][1])
a.shape, b.shape

((2784, 3, 1, 30), (2784,))

In [20]:
train_set, test_set = train_test_split(dataset, train_size=0.8)
len(train_set), len(test_set)

(2227, 557)

In [36]:
len(train_set[:][0])

2